In [1]:
import seaborn as sn
from systems import EpicActionRecogintionShapleyClassifier
from datasets.pickle_dataset import MultiPickleDataset
from models.esvs import V_MTRN, N_MTRN
import torch
from torch.utils.data import DataLoader
from pathlib import Path
import numpy as np
from frame_sampling import RandomSampler
import pandas as pd
from tqdm import tqdm
from collections import Counter

In [2]:
def no_collate(args):
    return args

trainloader = DataLoader(MultiPickleDataset('../datasets/epic-100/features/67217_train_features.pkl'), batch_size=1, collate_fn=no_collate, shuffle=True)
testloader = DataLoader(MultiPickleDataset('../datasets/epic-100/features/9668_val_features.pkl'), batch_size=1, collate_fn=no_collate, shuffle=False)

MemoryError: 

In [ ]:
v_models = [V_MTRN(frame_count=i, hidden_layer_size=2048, dropout_count=2) for i in range(1,9)]
n_models = [N_MTRN(frame_count=i, hidden_layer_size=2048, dropout_count=2) for i in range(1,9)]
frame_samplers = [RandomSampler(frame_count=i, snippet_length=1, test=False) for i in range(1,9)]
for i in range(len(v_models)):
    try:
        v_models[i].load_state_dict(torch.load(Path('../datasets/epic-100/models/') / f'sh_mtrn-type=verb-frames={i+1}-batch_size=512-lr=1e-05_hl=2048_dc=2.pt', map_location='cpu'))
    except FileNotFoundError:
        print(f'no verb model for {i+1} frames')
    try:
        n_models[i].load_state_dict(torch.load(Path('../datasets/epic-100/models/') / f'sh_mtrn-type=noun-frames={i+1}-batch_size=512-lr=1e-05_hl=2048_dc=2.pt', map_location='cpu'))
    except FileNotFoundError:
        print(f'no noun model for {i+1} frames')

In [ ]:
verbs = pd.read_csv('../datasets/epic-100/labels/EPIC_100_verb_classes.csv')['key'].to_dict()
nouns = pd.read_csv('../datasets/epic-100/labels/EPIC_100_noun_classes.csv')['key'].to_dict()

In [ ]:
def get_per_class_accuracy(model, sampler):
    
    def compute_accuracy(preds, labels) -> float:
        return float((labels == preds).sum()) / len(labels)

    if isinstance(model, V_MTRN):
        classes = verbs
        m_type="verb"
    else:
        classes = nouns
        m_type="noun"

    classifier = EpicActionRecogintionShapleyClassifier(
        model,
        torch.device("cpu"),
        None,
        sampler,
        sampler,
        trainloader,
        trainloader,
        m_type
    )
    
    correct_pred = {classname: 0 for classname in classes.keys()}
    incorrect_pred = {classname: {class_n: 0 for class_n in classes.keys()} for classname in classes.keys()}
    total_pred = {classname: 0 for classname in classes.keys()}
    
#     print(correct_pred)
#     print(incorrect_pred)
#     print(total_pred)
    
    model.eval()
    with torch.no_grad():
        
        validation_results = {
            'loss': [],
            'accuracy': [],
            'preds': [],
            'labels': []
        }
            
        for idx, data in tqdm(enumerate(trainloader),total=len(trainloader),dynamic_ncols=True):
            
            batch, labels = classifier._sample_frames(data)
            
            logits = model(batch.to(torch.device('cpu')))
            preds = logits.argmax(-1).cpu().numpy()
            label_classes = labels[f'{m_type}_class']
            
#             print(f'pred: {preds}, label: {label_classes.cpu().numpy()}')

            for label, pred in zip(label_classes.cpu().numpy(), preds):
                if label == pred:
                    correct_pred[label] += 1
                incorrect_pred[label][pred] += 1
                total_pred[label] += 1
                        
            validation_results['preds'].extend(list(preds))
            validation_results['labels'].extend(list(label_classes.cpu().numpy()))
            
    validation_results['accuracy'] = compute_accuracy(
        np.array(validation_results["labels"]), 
        np.array(validation_results["preds"])
    )
        
        
    print(validation_results['accuracy'])
    
#     print(correct_pred)
#     print(incorrect_pred)
#     print(total_pred)
    
    for classname, correct in correct_pred.items():
        try:
            accuracy = 100 * float(correct) / total_pred[classname]
            print("Accuracy for class {:5s} is: {:.1f}%".format(classes[classname], accuracy))
        except ZeroDivisionError:
            print("Did not predict class {:5s}".format(classes[classname]))
            
    return classes, incorrect_pred

In [ ]:
classes, incorrect_pred = get_per_class_accuracy(model=v_models[3], sampler=frame_samplers[3])

In [ ]:
class_list = list(classes.values())
cnf = np.array([[x for x in y.values()] for y in incorrect_pred.values()])

In [ ]:
import matplotlib.pyplot as plt

df_cm = pd.DataFrame(cnf, index = [i for i in class_list],
                  columns = [i for i in class_list])
plt.figure(figsize = (20,14))
cnf_matrix = sn.heatmap(df_cm, annot=False, cmap="YlGnBu")

In [ ]:
cnf_matrix.get_figure().savefig('conf.png', dpi=72)